In [87]:
from PIL import Image
import os
import pandas as pd
import numpy as np
import torch
from transformers import CLIPProcessor, CLIPModel
import time

In [88]:
def get_image_paths(directory):
    allowed_extensions = {".jpg", ".jpeg", ".png"}
    image_paths = [os.path.join(directory, filename) for filename in os.listdir(directory) if os.path.splitext(filename)[1].lower() in allowed_extensions]
    return image_paths

In [89]:
def create_df(image_paths):
    image_list = []
    for path in image_paths:
        image = Image.open(path)
        image_name = os.path.basename(path)
        exifdata = image._getexif()
        date_time, localisation = None, None
        if exifdata:
            for tag_id, value in exifdata.items():
                tag = Image.ExifTags.TAGS.get(tag_id, tag_id)
                if tag == "DateTime":
                    date_time = value
                elif tag == "GPSInfo":
                    localisation = value

            image_list.append((image_name, path, date_time, localisation))

        else:
            print("Aucune donnée EXIF trouvée.")

    df = pd.DataFrame(image_list, columns=["image_name", "path", "date_time", "localisation"])
    df["keywords"] = ""
    df["categories"] = ""

    return df

In [90]:
directory = "temp"
image_paths = get_image_paths(directory)
device = "cuda" if torch.cuda.is_available() else "cpu"

In [91]:
def image_embedding(paths):
    clip_model = CLIPModel.from_pretrained("laion/CLIP-ViT-L-14-laion2B-s32B-b82K").to(device)
    clip_processor = CLIPProcessor.from_pretrained("laion/CLIP-ViT-L-14-laion2B-s32B-b82K")

    images = []

    for path in paths:
        image = Image.open(path).convert("RGB")
        images.append(image)

    # Prétraitement des images en batch
    image_inputs = clip_processor(images=images, return_tensors="pt", padding=True).to(device)

    with torch.no_grad():
        image_embeddings = clip_model.get_image_features(**image_inputs)

    # Normalisation
    image_embeddings = image_embeddings / image_embeddings.norm(p=2, dim=-1, keepdim=True)
    image_embeddings = image_embeddings.cpu().numpy()

    return image_embeddings


In [108]:
start_time = time.time()

embedded_images = image_embedding(image_paths)

In [119]:
def sequential_clustering(embeddings, paths, threshold=0.6, n_neighbors=3):
    image_names = [os.path.basename(path) for path in paths]
    N = len(image_names)
    clusters = {}
    cluster_id = 0
    current_cluster = []

    for i in range(N):
        current_image = image_names[i]
        has_strong_link = False

        for j in range(1, n_neighbors + 1):
            if i + j < N:
                sim = np.dot(embeddings[i], embeddings[i + j])
                if sim > threshold:
                    has_strong_link = True
                    break

        current_cluster.append(current_image)

        if not has_strong_link:
            clusters[f"cluster_{cluster_id}"] = current_cluster
            cluster_id += 1
            current_cluster = []

    # Gère le dernier cluster s'il reste des images non assignées
    if current_cluster:
        clusters[f"cluster_{cluster_id}"] = current_cluster

    return clusters

In [120]:
clusters = sequential_clustering(embedded_images, image_paths)

In [121]:
print(clusters)

{'cluster_0': ['IMG_20250105_144147.jpg', 'IMG_20250105_144436.jpg', 'IMG_20250105_144540.jpg', 'IMG_20250105_144750.jpg', 'IMG_20250105_145543_BURST1.jpg', 'IMG_20250105_150144.jpg'], 'cluster_1': ['IMG_20250105_161636.jpg', 'IMG_20250105_165438.jpg', 'IMG_20250105_165450.jpg', 'IMG_20250105_165457.jpg'], 'cluster_2': ['IMG_20250107_180316.jpg'], 'cluster_3': ['IMG_20250108_143517.jpg'], 'cluster_4': ['IMG_20250109_174849.jpg'], 'cluster_5': ['IMG_20250110_163528.jpg'], 'cluster_6': ['IMG_20250110_164345.jpg'], 'cluster_7': ['IMG_20250111_152149.jpg', 'IMG_20250111_152150.jpg', 'IMG_20250111_154046.jpg'], 'cluster_8': ['IMG_20250113_201010.jpg'], 'cluster_9': ['IMG_20250114_193654.jpg'], 'cluster_10': ['IMG_20250119_131124_BURST1.jpg', 'IMG_20250119_131214_BURST1.jpg'], 'cluster_11': ['IMG_20250119_131317.jpg'], 'cluster_12': ['IMG_20250119_131334_BURST1.jpg', 'IMG_20250119_131335.jpg', 'IMG_20250119_131815_BURST1.jpg'], 'cluster_13': ['IMG_20250122_093940.jpg'], 'cluster_14': ['IMG_2

In [122]:
for cluster_id, images in clusters.items():
    for image in images:
        print(f"{cluster_id} : {image}")
    print("\n")

cluster_0 : IMG_20250105_144147.jpg
cluster_0 : IMG_20250105_144436.jpg
cluster_0 : IMG_20250105_144540.jpg
cluster_0 : IMG_20250105_144750.jpg
cluster_0 : IMG_20250105_145543_BURST1.jpg
cluster_0 : IMG_20250105_150144.jpg


cluster_1 : IMG_20250105_161636.jpg
cluster_1 : IMG_20250105_165438.jpg
cluster_1 : IMG_20250105_165450.jpg
cluster_1 : IMG_20250105_165457.jpg


cluster_2 : IMG_20250107_180316.jpg


cluster_3 : IMG_20250108_143517.jpg


cluster_4 : IMG_20250109_174849.jpg


cluster_5 : IMG_20250110_163528.jpg


cluster_6 : IMG_20250110_164345.jpg


cluster_7 : IMG_20250111_152149.jpg
cluster_7 : IMG_20250111_152150.jpg
cluster_7 : IMG_20250111_154046.jpg


cluster_8 : IMG_20250113_201010.jpg


cluster_9 : IMG_20250114_193654.jpg


cluster_10 : IMG_20250119_131124_BURST1.jpg
cluster_10 : IMG_20250119_131214_BURST1.jpg


cluster_11 : IMG_20250119_131317.jpg


cluster_12 : IMG_20250119_131334_BURST1.jpg
cluster_12 : IMG_20250119_131335.jpg
cluster_12 : IMG_20250119_131815_BURST1.jpg